<a href="https://colab.research.google.com/github/MJ199999/SW-Project/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 9.8 MB/s 
     |████████████████████████████████| 120 kB 65.2 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 


In [3]:
import pandas as pd

df = pd.read_excel('/gdrive/MyDrive/Colab Notebooks/sw-project/SW-Project/poem_key.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  292 non-null    int64 
 1   시인          292 non-null    object
 2   제목          292 non-null    object
 3   시           292 non-null    object
 4   키워드 전체      290 non-null    object
 5   키워드1        290 non-null    object
 6   키워드2        290 non-null    object
 7   키워드3        290 non-null    object
 8   키워드4        290 non-null    object
 9   키워드5        290 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.9+ KB


In [5]:
df.drop('Unnamed: 0', axis=1)

,시인,제목,시,키워드 전체,키워드1,키워드2,키워드3,키워드4,키워드5
0,강은교,우리가 물이 되어,우리가 물이 되어 만난다면\n가문 어느 집에선들 좋아하지 않으랴.\n우리가 키 큰 ...,"강물, 처녀, 나무, 가문, 기도",강물,처녀,나무,가문,기도
1,김춘수,꽃을 위한 서시,나는 시방 위험한 짐승이다.\n나의 손이 닿으면 너는 \n미지의 까마득한 어둠이 된...,"어둠, 신부, 울음, 짐승, 추억",어둠,신부,울음,짐승,추억
2,김춘수,꽃,내가 그의 이름을 불러 주기 전에는\n그는 다만\n하나의 몸짓에 지나지 않았다. \...,"향기, 무엇, 빛깔, 모두, 눈짓",향기,무엇,빛깔,모두,눈짓
3,김춘수,능금,그는 그리움에 산다.\n그리움은 익어서 \n스스로도 견디기 어려운\n빛깔이 되고 향...,"그리움, 축제, 충실, 향기, 바다",그리움,축제,충실,향기,바다
4,김춘수,부두에서,바다에 굽힌 사나이들\n하루의 노동을 끝낸\n저 사나이들의 억센 팔에 안긴\n깨지지...,"상어, 노동, 온전, 바다, 물개",상어,노동,온전,바다,물개
...,...,...,...,...,...,...,...,...,...
287,신석정,전아사,포옹할 꽃 한 송이 없는 세월을\n얼룩진 역사의 찢긴 자락에 매달려\n그대로 소스라...,"어둠, 억만, 가을비, 포옹, 음악",어둠,억만,가을비,포옹,음악
288,신석정,대바람 소리,대바람 소리\n들리더니\n소소한 대바람 소리\n창을 흔들더니\n\n소설 지낸 하늘을...,"구름, 소설, 제왕, 향기, 금은",구름,소설,제왕,향기,금은
289,송명희,나,"나, 가진 재물 없으나\n나, 남이 가진 지식 없으나\n나, 남에게 있는 건강 있지...","사랑, 지식, 하나님, 재물, 음성",사랑,지식,하나님,재물,음성
290,송수권,산문에 기대어,누이야\n가을 산 그리메에 빠진 눈썹 두어 낱을\n지금도 살아서 보는가\n정정한 눈...,"물방울, 눈물, 고뇌, 가을, 물고기",물방울,눈물,고뇌,가을,물고기


In [6]:
df = df.dropna()

In [7]:
train_dataset = df
# train_dataset = df[0:int(len(df)*0.9)]
# val_dataset = df[int(len(df)*0.9):]

In [8]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5", bos_token='</s>', eos_token='</s>', pad_token='<pad>')
gpt3_model = TFAutoModelForCausalLM.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.20.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.22.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.23.attn.masked_bias', 'transformer.h.15.attn.masked_bias', 'lm_head.weight', 'transformer.h.14.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.21.attn.masked_bias']
-

In [9]:
print(tokenizer.encode(df['시'].loc[0]))
print(tokenizer.tokenize(df['시'].loc[0]))
print(tokenizer.decode(tokenizer.encode(df['시'].loc[0])))

[31171, 36238, 30726, 33577, 30298, 376, 18792, 22688, 30696, 30198, 31634, 21068, 50647, 35706, 21788, 30005, 25624, 30417, 30697, 30397, 32051, 25512, 30279, 34310, 376, 25624, 42027, 30044, 42027, 30031, 31366, 42859, 51138, 30298, 30005, 376, 29880, 31570, 33401, 24196, 30121, 22692, 20160, 25404, 376, 25960, 32086, 31241, 37797, 30092, 34682, 39172, 376, 26213, 25768, 32051, 46222, 50272, 30312, 35673, 30005, 25260, 34551, 30865, 40981, 37254, 23272, 34509, 25628, 47102, 35306, 20476, 30298, 30005, 376, 36225, 19968, 30328, 31203, 376, 23280, 21956, 31082, 21900, 30589, 23028, 24784, 565, 24471, 25820, 30164, 35031, 34264, 376, 24224, 31517, 30093, 36902, 37556, 48481, 21076, 19016, 37563, 20528, 30005, 376, 22324, 30284, 40624, 41204, 43205, 25428, 376, 25960, 30093, 30288, 32767, 40659, 29880, 30688, 30134, 21956, 31082, 34257, 29152, 24644, 24644, 31793, 24644, 24644, 30093, 32198, 30248, 42859, 30023, 30441, 376, 25492, 31296, 40455, 29979, 27588, 376, 20091, 19016, 43308, 318

In [10]:
max_seq_len = 200

In [11]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=5, value=tokenizer.pad_token_id, padding='post')[0]

        output_tokens = bos_token + tokenizer.tokenize(label) + eos_token
        output_id = tokenizer.convert_tokens_to_ids(output_tokens)
        output_id = pad_sequences([output_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
        
        assert len(input_id) == 5, "Error with inpxut length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(output_id)

    input_ids = np.array(input_ids, dtype=int).reshape((-1,1))
    data_labels = np.array(data_labels, dtype=int).reshape((-1,1))

    return input_ids, data_labels

In [12]:
train_X_1, train_y = convert_examples_to_features(train_dataset['키워드1'], train_dataset['시'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 290/290 [00:00<00:00, 1142.80it/s]


In [13]:
train_X_2, train_y = convert_examples_to_features(train_dataset['키워드2'], train_dataset['시'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 290/290 [00:00<00:00, 1192.58it/s]


In [14]:
train_X_3, train_y = convert_examples_to_features(train_dataset['키워드3'], train_dataset['시'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 290/290 [00:00<00:00, 1209.15it/s]


In [15]:
train_X_4, train_y = convert_examples_to_features(train_dataset['키워드4'], train_dataset['시'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 290/290 [00:00<00:00, 1262.76it/s]


In [16]:
train_X_5, train_y = convert_examples_to_features(train_dataset['키워드5'], train_dataset['시'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 290/290 [00:00<00:00, 1207.49it/s]


In [17]:
gpt3_model.transformer.wte.trainable = False

In [18]:
gpt3_model.transformer.h[0].trainable = False
gpt3_model.transformer.h[1].trainable = False
gpt3_model.transformer.h[2].trainable = False
gpt3_model.transformer.h[3].trainable = False
gpt3_model.transformer.h[4].trainable = False
gpt3_model.transformer.h[5].trainable = False
gpt3_model.transformer.h[6].trainable = False
gpt3_model.transformer.h[7].trainable = False
gpt3_model.transformer.h[8].trainable = False
gpt3_model.transformer.h[9].trainable = False
gpt3_model.transformer.h[10].trainable = False

In [19]:
import tensorflow as tf
optimizer = tf.keras.optimizers.Adagrad(lr=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
gpt3_model.compile(optimizer=optimizer, loss=loss)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adagrad.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adagrad, self).__init__(name, **kwargs)


In [20]:
gpt3_model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 1162556160
 r)                                                              
                                                                 
Total params: 1,162,556,160
Trainable params: 577,372,800
Non-trainable params: 585,183,360
_________________________________________________________________


In [21]:
import torch
torch.cuda.empty_cache()

In [22]:
import gc
gc.collect()

64

# callbacks

In [23]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', # 모니터링 대상
    patience=3,        # 대상 기간동안 유지
    factor=0.5,         # 줄이는 양  
    verbose=1,                            
    min_learning_rate=0.00001)     # 최소 학습율


In [24]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [25]:
gpt3_model.fit(train_X_1, train_y, epochs=100, batch_size=4, validation_split=0.1, callbacks=[reduce_lr, early_stopping])
gpt3_model.fit(train_X_2, train_y, epochs=100, batch_size=4, validation_split=0.1, callbacks=[reduce_lr, early_stopping])
gpt3_model.fit(train_X_3, train_y, epochs=100, batch_size=4, validation_split=0.1, callbacks=[reduce_lr, early_stopping])
gpt3_model.fit(train_X_4, train_y, epochs=100, batch_size=4, validation_split=0.1, callbacks=[reduce_lr, early_stopping])
gpt3_model.fit(train_X_5, train_y, epochs=100, batch_size=4, validation_split=0.1, callbacks=[reduce_lr, early_stopping])

Epoch 1/100
327/327 [==============================] - 75s 114ms/step - loss: 5.6625 - val_loss: 3.9115 - lr: 0.0100
Epoch 2/100
327/327 [==============================] - 31s 95ms/step - loss: 4.9849 - val_loss: 3.6901 - lr: 0.0100
Epoch 3/100
327/327 [==============================] - 31s 95ms/step - loss: 4.8490 - val_loss: 3.5631 - lr: 0.0100
Epoch 4/100
327/327 [==============================] - 31s 95ms/step - loss: 4.7685 - val_loss: 3.4290 - lr: 0.0100
Epoch 5/100
327/327 [==============================] - 31s 94ms/step - loss: 4.7119 - val_loss: 3.5820 - lr: 0.0100
Epoch 6/100
327/327 [==============================] - 31s 95ms/step - loss: 4.6740 - val_loss: 3.4599 - lr: 0.0100
Epoch 7/100
327/327 [==============================] - ETA: 0s - loss: 4.6356
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
327/327 [==============================] - 31s 95ms/step - loss: 4.6356 - val_loss: 3.6376 - lr: 0.0100
Epoch 8/100
327/327 [=========================

In [26]:
max_key_len=5

In [27]:
keyword = "강물"

bos_token = [tokenizer.bos_token]
eos_token = [tokenizer.eos_token]

tokens = bos_token + tokenizer.tokenize(keyword) + eos_token
input_id = tokenizer.convert_tokens_to_ids(tokens)
input_id = pad_sequences([input_id], maxlen=max_key_len, value=tokenizer.pad_token_id, padding='post')[0]

input_id = np.array(input_id, dtype=int).reshape((-1,1))

input_ids1 = tokenizer(keyword, return_tensors='pt').input_ids
#input_ids1.unsqueeze(0)

In [28]:
gen_id = gpt3_model.generate(input_id, do_sample=False, min_length = 15, max_length=50, repetition_penalty=2.0, top_k=50, top_p=0.92,temperature=0.9, no_repeat_ngram_size=3)
gen_text = tokenizer.batch_decode(gen_id, skip_special_tokens=True)

In [29]:
gen_text

['\n \n이시다.\n.\n의가에그로다. 그고 나의은을 이름을도아푸이다.\n저 되고 않는깔에\n 바한,\n나의,\t들운 불인다면 그의 눈여 나는 눈어월 되어 불러',
 '강\n \n이시다.\n.\n의가에그로다. 그고 나의을은 이름을도아푸이다.\n저 되고 않는깔에\n 바한,\n나의,\t들운다면 불인 그의 눈여 나는  되어 익어눈',
 '물\n \n이시다.\n.\n의가에그로다. 그고 나의은을 이름을도아푸이다.\n저 되고 않는깔에\n 바한,\n나의,\t들운다면 불인 그의 눈여 나는  익어 되어 여',
 '\n \n이시다.\n.\n의가에그로다. 그고 나의은을 이름을도아푸이다.\n저 되고 않는깔에\n 바한,\n나의,\t들운 불인다면 그의 눈여 나는 눈어월 되어 불러',
 '\n \n이시다.\n.\n의가에그로다. 그고 나의은을 이름을도아푸이다.\n저 되고 않는깔에\n 바한,\n나의,\t들운다면 불인 그의 눈여 나는 눈어월 되어 불러']

In [ ]:
import os
model_name = "skt-ko-gpt-trinity-1.2B-v0.5-4"
save_path = os.path.join('/gdrive/MyDrive/Colab Notebooks/sw-project/SW-Project/', model_name)
os.makedirs(save_path)
gpt3_model.save_pretrained(save_path)